# Understanding and Building the Autoencoder

Autoencoders are neural networks designed for unsupervised learning tasks, primarily for dimensionality reduction for feature learning. They work by compressing the input into a latent-space representation, and then rescontructing the output from this representation. 

### How Autoencoders simulate the brain functions:

Autoencoders can simulate how certain neurons in the brain might be responsible for detecting specific features such as eyes, nose and mouth by:

1. Encoding Phase: Learning a compressed representation of the face, which might be thought of as how the brain encodes visual inputs into a more abstract representation.
2. Decoding Phase: Attempting to reconstruct the original image from this compressed representation, similar to how the brain might reconstruct visual information from abstracted signals.

### Implementing the autoencoder with VGG-Face
1. Feature extraction with VGG-Face
2. Building the Decoder
3. Training the Autoencoder
4. Simulating the neurological deficits

We will be using the VGG-Face as the encoder and a mirrored architecture as the decoder. 

In [ ]:
# Import required librabries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras_vggface.vggface import VGGFace
from keras.layers import Input, Dense, Flatten, Reshape, Dropout, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Step 1: Load and Prepare the data

In [ ]:
# Helper functions to load and preprocess images
def load_images(image_paths, target_size):
    images = []
    labels= []
    for img_path in image_paths:
        img = image.load_img(img_path, target_size=target_size)
        img = image.img_to_array(img)
        img = np.expand_dims(img, axis=0)
        img = img / 255.0 # normalize the image pixels
        images.append(img)
    return np.vstack(images)

In [ ]:
# Variables for normal_faces, blurred_eyes, blurred_nose, and blurred_mouth
from create-datasets import image_paths, blurred_eyes_paths, blurred_nose_paths, blurred_mouth_path
print(len(image_paths))
print(len(blurred_eyes_paths))
print(len(blurred_nose_paths))
print(len(blurred_mouth_paths))

In [ ]:
# Load images 
X_normal = load_images(image_paths, target_size=(224, 224))
y_normal = np.array(labels) # Ensure labels are set correctly as 0 or 1 for binary classification (face/no face)

# Split the data
X_normal_train, X_normal_test, y_normal_train, y_normal_test = train_test_split(X_normal, y_normal, test_size=0.2, random_state=42)

In [ ]:
# For blurred eyes
X_eyes = load_images(blurred_eyes_paths, target_size=(224, 224))
y_eyes = np.array(labels)

# Split the data
X_eyes_train, X_eyes_test, y_eyes_train, y_eyes_test = train_test_split(X_eyes, y_eyes, test_size=0.2, random_state=42)

In [ ]:
# For blurred nose
X_nose = load_images(blurred_nose_paths, target_size=(224, 224))
y_nose = np.array(labels)

# Split the data
X_nose_train, X_nose_test, y_nose_train, y_nose_test = train_test_split(X_nose, y_nose, test_size=0.2, random_state=42)

In [ ]:
# For blurred mouth
X_mouth = load_images(blurred_mouth_paths, target_size=(224, 224))
y_mouth = np.array(labels)

# Split the data
X_mouth_train, X_mouth_test, y_mouth_train, y_mouth_test = train_test_split(X_mouth, y_mouth, test_size=0.2, random_state=42)

# Step 2: Build the Autoencoder with VGG-Face as Encoder

In [ ]:
def build_autoencoder():
    # Encoder
    vggface = VGGFace(model='vgg16', include_top=False, input_shape=(224, 224, 3), pooling='max')
    for layer in vggface.layers:
        layer.trainable = False
    
    # Decoder
    flattened = Flatten()(vggface.output)
    dense = Dense(512, activation='relu')(flattened)
    reshaped = Reshape((8, 8, 8))(dense)
    upsample1 = Conv2DTranspose(128, (3, 3), activation='relu', strides=(2, 2), padding='same')(reshaped)
    upsample2 = Conv2DTranspose(64, (3, 3), activation='relu', strides=(2, 2), padding='same')(upsample1)
    upsample3 = Conv2DTranspose(32, (3, 3), activation='relu', strides=(2, 2), padding='same')(upsample2)
    decoder = Conv2DTranspose(3, (3, 3), activation='sigmoid', padding='same')(upsample3)
    
    # Autoencoder
    autoencoder = Model(vggface.input, decoder)
    autoencoder.compile(optimizer='adam', loss='mse')
    
    return autoencoder

In [ ]:
autoencoder = build_autoencoder()

# Step 3: Train the Autoencoder

In [ ]:
autoencoder.fit(X_normal_train,
                X_normal_train, 
                epochs = 10, 
                batch_size = 32, 
                validation_data=(X_normal_test, X_normal_test))

# Step 4: Evaluate the model

To evaluate the model, we compute accuracy, precision, recall and F1-score

In [ ]:
# Get predictions
predictions = autoencoder.predict(X_test)
predictions = np.round(predictions).astype(int)

# Flatten the predictions and truth for metric calculations
y_true = X_test.flatten()
y_pred = predictions.flatten()

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f'Accuracy: {accuracy}\nPrecision: {precision}\nRecall: {recall}\nF1-Score: {f1}')

# Step 5: Simulate Neurological Deficits

In [ ]:
def deactivate_neurons(layer_name, indices):
    layer = autoencoder.get_layer(layer_name)
    weights, biases = layer.get_weights()
    weights[:, indices] = 0
    layer.set_weights([weights, biases])

deactivate_neurons('dense_1', [10, 20, 30])  # Example of deactivating certain neurons